# Imports

In [1]:
from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.applications import VGG19
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
from helper import DataLoader
import numpy as np
import os
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import pydot
from keras.callbacks import TensorBoard, ModelCheckpoint
from glob import glob
from tqdm import tqdm_notebook
import scipy
import imageio
from tensorlayer.prepro import *
import tensorlayer as tl

Using TensorFlow backend.


# Params

In [2]:
hr_shape = (224,224,3)
img_hr = Input(shape=hr_shape)
lr_shape = (56,56,3)
img_lr = Input(shape=lr_shape)

# Hyper Params

Instructions for updating:
Colocations handled automatically by placer.


# Generator

In [3]:
def build_generator(img_lr,lr_shape):
    def residual_block(layer_input, filters):
        """Residual block described in paper"""
        d = Conv2D(filters, kernel_size=3, strides=1,
                   padding='same')(layer_input)
        d = Activation('relu')(d)
        d = BatchNormalization(momentum=0.8)(d)
        d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(d)
        d = BatchNormalization(momentum=0.8)(d)
        d = Add()([d, layer_input])
        return d

    def deconv2d(layer_input):
        """Layers used during upsampling"""
        u = UpSampling2D(size=2)(layer_input)
        u = Conv2D(256, kernel_size=3, strides=1, padding='same')(u)
        u = Activation('relu')(u)
        return u

    # Low resolution image input
    img_lr = Input(shape=lr_shape)

    # Pre-residual block
    c1 = Conv2D(64, kernel_size=9, strides=1, padding='same')(img_lr)
    c1 = Activation('relu')(c1)

    # Propogate through residual blocks
    r = residual_block(c1, 64)
    for _ in range(16 - 1):
        r = residual_block(r, 64)

    # Post-residual block
    c2 = Conv2D(64, kernel_size=3, strides=1, padding='same')(r)
    c2 = BatchNormalization(momentum=0.8)(c2)
    c2 = Add()([c2, c1])

    # Upsampling
    u1 = deconv2d(c2)
    u2 = deconv2d(u1)

    # Generate high resolution output
    gen_hr = Conv2D(3, kernel_size=9, strides=1,
                    padding='same', activation='tanh')(u2)

    return Model(img_lr, gen_hr)

gen = build_generator(img_lr,lr_shape)

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
SVG(model_to_dot(gen, show_layer_names=True, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
gen.summary()

# Discriminator

In [4]:
def build_discriminator(hr_shape):

        def d_block(layer_input, filters, strides=1, bn=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=3, strides=strides,
                       padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        # Input img
        d0 = Input(shape=hr_shape)

        d1 = d_block(d0, 64, bn=False)
        d2 = d_block(d1, 64, strides=2)
        d3 = d_block(d2, 64*2)
        d4 = d_block(d3, 64*2, strides=2)
        d5 = d_block(d4, 64*4)
        d6 = d_block(d5, 64*4, strides=2)
        d7 = d_block(d6, 64*8)
        d8 = d_block(d7, 64*8, strides=2)

        d9 = Dense(64*16)(d8)
        d10 = LeakyReLU(alpha=0.2)(d9)
        validity = Dense(1, activation='sigmoid')(d10)

        return Model(d0, validity)
    
disc = build_discriminator(hr_shape)
disc.compile(loss='mean_squared_error',
             optimizer=optimizer,
             metrics=['binary_accuracy'])
disc.trainable = False

NameError: name 'optimizer' is not defined

In [ ]:
SVG(model_to_dot(gen, show_layer_names=True, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
disc.summary()

# VGG

In [6]:
def build_vgg(hr_shape):
    """
    Builds a pre-trained VGG19 model that outputs image features extracted at the
    third block of the model
    """
    vgg = VGG19(weights="imagenet")
    # Set outputs to outputs of last conv. layer in block 3
    # See architecture at: https://github.com/keras-team/keras/blob/master/keras/applications/vgg19.py
    vgg.outputs = [vgg.layers[9].output]

    img = Input(shape=hr_shape)

    # Extract image features
    img_features = vgg(img)

    return Model(img, img_features)

vgg = build_vgg(hr_shape)
vgg.compile(loss='mean_squared_error',
            optimizer=optimizer,
            metrics=['binary_accuracy'])
vgg.trainable = False

ResourceExhaustedError: OOM when allocating tensor with shape[25088,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node fc1/random_uniform/RandomUniform (defined at /home/ubuntu/anaconda3/envs/srgan/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4139) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'fc1/random_uniform/RandomUniform', defined at:
  File "/home/ubuntu/anaconda3/envs/srgan/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/srgan/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/srgan/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/anaconda3/envs/srgan/lib/python3.7/site-packages/trait

# Combined

In [5]:
gen_hr = gen(img_lr)
vgg_features = vgg(gen_hr)
validity = disc(gen_hr)
combined = Model([img_lr, img_hr], [validity, vgg_features])
combined.compile(loss=['binary_crossentropy', 'mean_squared_error'],
                              loss_weights=[1e-3, 1],
                              optimizer=optimizer)

NameError: name 'vgg' is not defined

In [ ]:
SVG(model_to_dot(combined, show_layer_names=True, show_shapes=True).create(prog='dot', format='svg'))

# Training

In [8]:
path = "./mainDataset"
dataset_name = "train"
steps = len(glob(path+'/%s/*' % (dataset_name)))//batchSize
# for the VGG feature out true labels
patch = int(224 / 2**4)
disc_patch = (patch, patch, 1)

In [9]:
train_hr_img_list = sorted(tl.files.load_file_list(path=path+'/%s/' % (dataset_name), regx='.*.png', printable=False))

train_hr_imgs = tl.vis.read_images(train_hr_img_list, path=path+'/%s/' % (dataset_name), n_threads=32)

[TL] read 32 from ./mainDataset/train/
[TL] read 64 from ./mainDataset/train/
[TL] read 96 from ./mainDataset/train/
[TL] read 128 from ./mainDataset/train/
[TL] read 160 from ./mainDataset/train/
[TL] read 192 from ./mainDataset/train/
[TL] read 224 from ./mainDataset/train/
[TL] read 256 from ./mainDataset/train/
[TL] read 288 from ./mainDataset/train/
[TL] read 320 from ./mainDataset/train/
[TL] read 352 from ./mainDataset/train/
[TL] read 384 from ./mainDataset/train/
[TL] read 416 from ./mainDataset/train/
[TL] read 448 from ./mainDataset/train/
[TL] read 480 from ./mainDataset/train/
[TL] read 512 from ./mainDataset/train/
[TL] read 544 from ./mainDataset/train/
[TL] read 576 from ./mainDataset/train/
[TL] read 608 from ./mainDataset/train/
[TL] read 640 from ./mainDataset/train/
[TL] read 672 from ./mainDataset/train/
[TL] read 704 from ./mainDataset/train/
[TL] read 736 from ./mainDataset/train/
[TL] read 768 from ./mainDataset/train/
[TL] read 800 from ./mainDataset/train/
[TL

In [10]:
tensorboard = TensorBoard(
  log_dir='log/pretrain_fix/run11',
  histogram_freq=0,
  batch_size=batchSize,
  write_graph=True,
  write_grads=True
)
tensorboard.set_model(combined)

In [ ]:
#gen.load_weights("checkpoints/gen_init.h5")

In [11]:
def datagen(dev_hr_imgs,batchSize,is_testing=False):
    while(True):
        imgs_hr=[]
        imgs_lr=[]
        imgs = np.random.choice(dev_hr_imgs,batchSize)
        img_hr = tl.prepro.threading_data(imgs, fn=crop, wrg=224, hrg=224, is_random=True)
        img_lr = tl.prepro.threading_data(img_hr, fn=imresize,size=[56, 56], interp='bicubic', mode=None)
    
        imgs_hr = np.array(img_hr) / 127.5 - 1.
        imgs_lr = np.array(img_lr) / 127.5 - 1.

        yield imgs_hr, imgs_lr

In [12]:
datagenObj = datagen(train_hr_imgs,batchSize)

In [13]:
sample_hr = tl.prepro.threading_data(train_hr_imgs[0:batchSize], fn=crop, wrg=224, hrg=224, is_random=True)
sample_lr = tl.prepro.threading_data(sample_hr, fn=imresize,size=[56, 56], interp='bicubic', mode=None)

In [14]:
tl.vis.save_images(sample_hr, [int(ni), int(ni)],'images/'+dataset_name+'/sample_hr.png')
tl.vis.save_images(sample_lr, [int(ni), int(ni)],'images/'+dataset_name+'/sample_lr.png')

## Pretrain Gen

In [15]:
for epoch in range(10):
    # Sample images and their conditioning counterparts
    imgs_hr, imgs_lr = next(datagenObj)

    # From low res. image generate high res. version
    fake_hr = gen.predict(imgs_lr)

    valid = np.ones((batchSize,) + disc_patch)
    fake = np.zeros((batchSize,) + disc_patch)

    # Train the discriminators (original images = real / generated = Fake)
    disc.trainable = True
    d_loss_real = disc.train_on_batch(imgs_hr, valid)
    d_loss_fake = disc.train_on_batch(fake_hr, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    disc.trainable = False

Instructions for updating:
Use tf.cast instead.


In [20]:
for epoch in range(100):
    print("Epoch: "+str(epoch))
    for step in tqdm_notebook(range(0,steps)):    
        # ------------------
        #  Train Generator
        # ------------------

        # Sample images and their conditioning counterparts
        imgs_hr, imgs_lr = next(datagenObj)

        # The generators want the discriminators to label the generated images as real
        valid = np.ones((batchSize,) + disc_patch)

        # Extract ground truth image features using pre-trained VGG19 model
        image_features = vgg.predict(imgs_hr)

        # Train the generators
        g_loss = combined.train_on_batch([imgs_lr, imgs_hr], [valid, image_features])
        
        tb_step = step + int(epoch*(steps))
        tensorboard.on_epoch_end(tb_step, {"g_int_loss":g_loss[0],"g_mse":g_loss[1]})
        
        
    if epoch % 2 == 0:
        out = gen.predict(sample_lr)
        tl.vis.save_images(out, [int(ni), int(ni)],'images/'+dataset_name+'/A_train_%d.png' % int(epoch))
    gen.save_weights("./checkpoints/gen_init.h5")
tensorboard.on_train_end(None)

Epoch: 0



Epoch: 1



Epoch: 2



Epoch: 3



Epoch: 4


...


Epoch: 95



Epoch: 96



Epoch: 97



Epoch: 98



Epoch: 99


Epoch:0



Epoch:1



Epoch:2



Epoch:3



Epoch:4


...


Epoch:266



Epoch:267



Epoch:268



Epoch:269



Epoch:270


In [38]:
[0].shape

(14, 14, 1)